<h2>Load all data we need from all datasets. Get all the columns and things we need. Save cleaned dfs to use to make d3 vis

In [7]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
import plotly.express as px
import geopandas as gpd
import requests


<h3>Load Data

In [45]:
API_TOKEN = 'tRPy6aES2cCa26Y6QSvwPzC8eswqiiVFPGAJ'
BASE_URL = "https://api.iucnredlist.org/api/v4"

In [68]:
def get_species_by_group(class_name):
    """
    Fetch all species for a taxonomic group (e.g., 'Amphibia')
    Returns a list of species dictionaries
    """
    url = f"{BASE_URL}/taxa/class/{class_name}"
    headers = {
        "Authorization": API_TOKEN,
        "Accept": "application/json"
    }
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print("Error:", response.status_code)
        return []
    
    data = response.json()
    return data.get('assessments', [])

def get_threats(sis_id):
    url = f"{BASE_URL}/threats/species/{sis_id}"
    headers = {"Authorization": API_TOKEN, "Accept": "application/json"}
    response = requests.get(url, headers=headers)
    return response.json().get("threats", [])


def get_assessments(assessment_id):
    url = f"{BASE_URL}/assessment/{assessment_id}"
    headers = {"Authorization": API_TOKEN, "Accept": "application/json"}
    response = requests.get(url, headers=headers)
    return response.json()



In [52]:
#amphibians
amphibians = get_species_by_group("Amphibia")
print(f"Fetched {len(amphibians)} amphibian species")
print(amphibians[:2])  # preview first 2

#reptiles
reptiles = get_species_by_group("Reptilia")
print(f"Fetched {len(reptiles)} reptile species")
print(reptiles[:2])  # preview first 2

Fetched 100 amphibian species
[{'year_published': '2020', 'latest': True, 'possibly_extinct': False, 'possibly_extinct_in_the_wild': False, 'sis_taxon_id': 12776, 'url': 'https://www.iucnredlist.org/species/12776/508612', 'taxon_scientific_name': 'Mantella aurantiaca', 'red_list_category_code': 'EN', 'assessment_id': 508612, 'scopes': [{'description': {'en': 'Global'}, 'code': '1'}]}, {'year_published': '2016', 'latest': False, 'possibly_extinct': False, 'possibly_extinct_in_the_wild': False, 'sis_taxon_id': 1301, 'url': 'https://www.iucnredlist.org/species/1301/511335', 'taxon_scientific_name': 'Anhydrophryne rattrayi', 'red_list_category_code': 'VU', 'assessment_id': 511335, 'scopes': [{'description': {'en': 'Global'}, 'code': '1'}]}]
Fetched 100 reptile species
[{'year_published': '2021', 'latest': True, 'possibly_extinct': False, 'possibly_extinct_in_the_wild': False, 'sis_taxon_id': 10041, 'url': 'https://www.iucnredlist.org/species/10041/495907', 'taxon_scientific_name': 'Heosemy

In [78]:
def create_df(animal_json):
    rows = []
    for species in animal_json:
        assessment = get_assessments(species["assessment_id"])
        
        if isinstance(assessment, dict) and "result" in assessment:
            assessment = assessment["result"][0]

        # Safely extract threats
        threats_list = assessment.get("threats", []) or []
        threats = [
            (t.get("description", {}).get("en") if isinstance(t.get("description"), dict)
             else t.get("description"))
            for t in threats_list
        ]

        # Safely extract habitats
        habitat_list = assessment.get("habitats", []) or []
        habitats = [
            (h.get("description", {}).get("en") if isinstance(h.get("description"), dict)
             else h.get("description"))
            for h in habitat_list
        ]

        # Safely extract population trend
        pop = assessment.get("population_trend", {})
        if isinstance(pop, dict):
            population_trend = (
                pop.get("description", {}).get("en")
                if isinstance(pop.get("description"), dict)
                else pop.get("description")
            )
        
        
        rows.append({
            "taxon":assessment['taxon']['class_name'],
            "species_name": species["taxon_scientific_name"],
            "red_list_category": species["red_list_category_code"],
            "threats": threats,
            "population_trend": population_trend,
            "habitats": habitats
        })
    return pd.DataFrame(rows)

In [75]:
amphibian_df = create_df(amphibians)

amphibian_df.columns
amphibian_df.shape
amphibian_df.head(3)

,taxon,species_name,red_list_category,threats,population_trend,habitats
0,AMPHIBIA,Mantella aurantiaca,EN,"[Intentional use (species is the target), Mini...",Decreasing,"[Forest - Subtropical/Tropical Swamp, Wetlands..."
1,AMPHIBIA,Anhydrophryne rattrayi,VU,"[Agro-industry plantations, Unspecified specie...",Unknown,"[Forest - Temperate, Grassland - Temperate]"
2,AMPHIBIA,Bokermannohyla itapoty,LC,[],Stable,[Grassland - Subtropical/Tropical High Altitud...


In [79]:
reptile_df = create_df(reptiles)

reptile_df.columns
reptile_df.shape
reptile_df.head(3)

,taxon,species_name,red_list_category,threats,population_trend,habitats
0,REPTILIA,Heosemys annandalii,CR,[Intentional use: (subsistence/small scale) [h...,Decreasing,[Wetlands (inland) - Seasonal/Intermittent Fre...
1,REPTILIA,Iguanognathus werneri,DD,[],Decreasing,[Unknown]
2,REPTILIA,Indotestudo elongata,CR,"[Scale Unknown/Unrecorded, Unintentional effec...",Decreasing,"[Shrubland - Subtropical/Tropical Dry, Forest ..."


In [80]:
combined_df = pd.concat([amphibian_df, reptile_df]).reset_index(drop=True)

combined_df.shape
combined_df.head()

,taxon,species_name,red_list_category,threats,population_trend,habitats
0,AMPHIBIA,Mantella aurantiaca,EN,"[Intentional use (species is the target), Mini...",Decreasing,"[Forest - Subtropical/Tropical Swamp, Wetlands..."
1,AMPHIBIA,Anhydrophryne rattrayi,VU,"[Agro-industry plantations, Unspecified specie...",Unknown,"[Forest - Temperate, Grassland - Temperate]"
2,AMPHIBIA,Bokermannohyla itapoty,LC,[],Stable,[Grassland - Subtropical/Tropical High Altitud...
3,AMPHIBIA,Pseudis platensis,LC,"[Agro-industry farming, Small-holder grazing, ...",Stable,[Wetlands (inland) - Seasonal/Intermittent Fre...
4,AMPHIBIA,Tsingymantis antitra,EN,"[Small-holder grazing, ranching or farming, In...",Unknown,[Wetlands (inland) - Permanent Rivers/Streams/...
